In [6]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import nbimporter
from diametre_monnaie import conversion_piece

In [7]:
image_path = '../DataBase/image7.jpg' 

reference_object_mm = 22.6  # diamètre de la pièce de monnaie en mm 

# Les dimensions réelles en millimètres de la piece 

diametre_reel_mm = 20  # Diamètre réel du filetage en mm  
pas_reel_mm = 25.4 / 14 # Pas réel du filetage en mm

In [8]:
def draw_tooth_pattern(image_path, pas_reel_mm, reference_object_mm, rotation):
    # Supposons que cette fonction retourne la taille en pixels du diamètre réel
    size_pixels = conversion_piece(image_path, reference_object_mm)
    
    if size_pixels <= 0:
        raise ValueError("La taille en pixels doit être positive et non nulle")

    pas_pixels = pas_reel_mm / size_pixels
    tooth_height = max(int(1 / size_pixels), 1)  # Assurez-vous que la hauteur est au moins 1
    
    pattern_width = int(pas_pixels) * 4  # Largeur pour 4 dents
    
    pattern_image = np.zeros((tooth_height, pattern_width), dtype=np.uint8)
    
    # Dessiner 4 dents
    for i in range(4):
        start_x = int(i * pas_pixels + pas_pixels * 0.25)
        end_x = int((i + 1) * pas_pixels - pas_pixels * 0.25)
        middle_x = int((start_x + end_x) / 2)
        
        cv2.line(pattern_image, (start_x, tooth_height - 1), (middle_x, 0), 255, 1)
        cv2.line(pattern_image, (middle_x, 0), (end_x, tooth_height - 1), 255, 1)

    if rotation:
        pattern_image = cv2.rotate(pattern_image, cv2.ROTATE_180)

    return pattern_image


In [9]:
pattern_image = draw_tooth_pattern(image_path,pas_reel_mm,reference_object_mm,False)
pattern_image_r = draw_tooth_pattern(image_path,pas_reel_mm,reference_object_mm,True)
cv2.imwrite('pattern_image.jpg', pattern_image)
cv2.imwrite('pattern_image_r.jpg', pattern_image_r)

True

In [10]:
def match(template,img):
    w, h = template.shape[::-1]
    res = cv2.matchTemplate(img,template,cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

    top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv2.rectangle(img,top_left, bottom_right, 255, 2)
    
    return img,res,max_val

In [16]:
def match(template,img):
    w, h = template.shape[::-1]
    res = cv2.matchTemplate(img,template,cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

    top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv2.rectangle(img,top_left, bottom_right, 255, 2)
    
    return img,res,max_val

def match_top_bottom(template,template_r, img_path, vertical_split_ratio=0.5):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    h, w = img.shape

    split_line = int(vertical_split_ratio * h)

    img_with_matches = img.copy()

    top_region = img[:split_line, :]
    bottom_region = img[split_line:, :]

    img_with_matches_h, res_top, max_val_top = match(template_r, top_region)
    
    _, _, _, max_loc_top = cv2.minMaxLoc(res_top)
    top_left_top = max_loc_top

    img_with_matches_b, res_bottom, max_val_bottom = match(template, bottom_region)
    
    _, _, _, max_loc_bottom = cv2.minMaxLoc(res_bottom)
    bottom_right_bottom = (max_loc_bottom[0] + template.shape[1], max_loc_bottom[1] + split_line + template.shape[0])

    output_path = 'matched_bottom_h.jpg'
    cv2.imwrite(output_path, img_with_matches_h)

    output_path = 'matched_bottom_b.jpg'
    cv2.imwrite(output_path, img_with_matches_b)
    
    image_fusionnee = np.vstack((img_with_matches_h, img_with_matches_b))
    
    output_path = 'matched.jpg'
    cv2.imwrite(output_path, image_fusionnee)
    return image_fusionnee, (res_top, res_bottom), (max_val_top, max_val_bottom),top_left_top,bottom_right_bottom


In [17]:
match_top_bottom(pattern_image,pattern_image_r, image_path)

(array([[143, 142, 140, ..., 162, 162, 162],
        [143, 142, 140, ..., 162, 162, 162],
        [143, 142, 141, ..., 162, 162, 162],
        ...,
        [155, 153, 151, ..., 182, 182, 182],
        [155, 152, 149, ..., 182, 182, 182],
        [155, 152, 149, ..., 182, 182, 182]], dtype=uint8),
 (array([[ 0.00478011, -0.00734098, -0.00743546, ..., -0.02885098,
          -0.02946723, -0.02861918],
         [ 0.01420025,  0.01031761, -0.00184617, ..., -0.02801819,
          -0.02879716, -0.02389637],
         [ 0.01605636,  0.02148383,  0.00323427, ..., -0.02774016,
          -0.0279217 , -0.02253641],
         ...,
         [-0.01400566,  0.00246581,  0.0111005 , ...,  0.02250917,
           0.03893349,  0.04804771],
         [-0.02316808, -0.00766008,  0.00996475, ...,  0.01748154,
           0.03177003,  0.04461168],
         [-0.02932882, -0.01834198, -0.00081755, ...,  0.01552644,
           0.02646689,  0.03720336]], dtype=float32),
  array([[ 0.02932685,  0.03957905,  0.04118163

In [ ]:
img, res, max_val = match(pattern_image, image_path)
cv2.imwrite('matched_image.jpg', img)